In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *

my_env = 'data'
my_key = get_key('koray_data')
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=FileProcessed"
all_pc_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_pc_resp))

482


In [11]:
# find all runs from 4dn lab without a contributing lab
files_with_source = [i for i in all_pc_resp if i.get('source_experiments')]
print(len(files_with_source))
## Patching all processed files
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
action = False
def get_attr_from_pc_file(file_resp):
    lab = ''
    award = ''
    source_exp = file_resp['source_experiments'][0]
    exp_info = ff_utils.get_metadata(source_exp, key=my_key, add_on='frame=raw')
    lab = exp_info['lab']
    award = exp_info['award']
    cont_labs = exp_info.get('contributing_labs', [])
    return lab, award, cont_labs

counter = 0
for a_file in files_with_source:
    counter += 1
    if counter % 100 == 0:
        print(counter)
    my_lab, my_award, my_cont_labs = get_attr_from_pc_file(a_file)
    if my_lab in my_cont_labs:
        print("PROBLEM, owner lab also in cont_labs", skipping)
        continue
    if FDlab not in my_cont_labs:
        my_cont_labs.append(FDlab)
    patch_data = {'lab':my_lab,
                  'award': my_award,
                  'contributing_labs':my_cont_labs}
    #print(patch_data)
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_file['uuid'], key=my_key)

0


In [3]:
# find all runs from 4dn lab
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=WorkflowRunAwsem"
all_awsem_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_awsem_resp))

1334


In [5]:
files_with_cont_and_lab_same = []
# add lab/award/contlab to wfrs
counter = 0
action = True
acted = 0
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter, acted)
    
    my_lab = ''
    my_award = ''
    my_cont_lab = ''
    patch_data = {}
    
    input_files = a_resp['input_files']
    run_on_expected = False
    for an_input in input_files:
        # if this wfr ran on processed files
        input_id = an_input['value']['@id']
        if input_id.startswith('/files-processed/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key=my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
        # if this wfr ran on raw files
        elif input_id.startswith('/files-fastq/') or input_id.startswith('/files-microscopy/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key = my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
    # I normally exclude files reference since they are part of many runs, and often belong to dcic
    # in a second round, if the file type is not in one if the expected ones, check what it is 
    # this case often convers md5 runs on reference files or other file types (eg filecalibration)
    if not run_on_expected:
        print('skipping wfr on', input_id)
        continue

    if my_lab == FDlab:
        continue
           
    # if file is not from FDN(user submitted pc), add it to cont labs
    if FDlab not in my_cont_lab:
        my_cont_lab.append(FDlab)  

    if my_lab in my_cont_lab:
        my_cont_lab.remove(my_lab)
        files_with_cont_and_lab_same.append(input_id)
        print(an_input['value']['accession'], 'has lab also in cont lab')
        
    patch_data = {'lab':my_lab,
                  'award': my_award}
    if my_cont_lab:
        patch_data['contributing_labs'] = my_cont_lab
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_resp['@id'], key=my_key)
        acted += 1



skipping wfr on /files-reference/4DNFI2493SDN/
skipping wfr on /files-reference/4DNFIMQPTYDY/
100 90
200 190
skipping wfr on /files-vistrack/4DNFIV99XB3G/
skipping wfr on /files-vistrack/4DNFII9ORPLP/
skipping wfr on /files-vistrack/4DNFIM966TBO/
skipping wfr on /files-vistrack/4DNFIOGFSCY1/
skipping wfr on /files-vistrack/4DNFIRTIE87S/
skipping wfr on /files-vistrack/4DNFIICPMGV8/
skipping wfr on /files-vistrack/4DNFILBZDC5F/
skipping wfr on /files-vistrack/4DNFI6PWAVGC/
skipping wfr on /files-vistrack/4DNFI796OM7C/
skipping wfr on /files-vistrack/4DNFINCISFAP/
skipping wfr on /files-vistrack/4DNFIQKQDZQT/
skipping wfr on /files-vistrack/4DNFI6N7NQ25/
skipping wfr on /files-vistrack/4DNFIZZ49XA3/
skipping wfr on /files-vistrack/4DNFI6DSRFA7/
skipping wfr on /files-vistrack/4DNFIHXE6C3Z/
skipping wfr on /files-vistrack/4DNFIEU6VG86/
skipping wfr on /files-vistrack/4DNFI8O4RIVQ/
skipping wfr on /files-vistrack/4DNFIETGL38A/
skipping wfr on /files-vistrack/4DNFIR5E7D6L/
skipping wfr on /

1300 1070
